<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="400" alt="cognitiveclass.ai logo">
</center>

# Investigation on MATIC/BUSD cryptocurrency based on EMA, Crypto Trading Metrics, WMA 

## Lab 6. Knowledge Check

Estimated time needed: **1** hour


### Objectives

After completing this lab you will be able to:

*   Be confident about your data analysis skills

<h3>Table of Contents</h3>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ol>
    <li><a href="https://#import_load__data">Import and Load Data</a></li>
    <li><a href="https://#questions">Questions</a></li>
        <ul>
            <li><a href="https://#question1">Question #1</a></li>
            <li><a href="https://#question2">Question #2</a></li>
            <li><a href="https://#question3">Question #3</a></li>
            <li><a href="https://#question4">Question #4</a></li>
            <li><a href="https://#question5">Question #5</a></li>
            <li><a href="https://#question6">Question #6</a></li>
            <li><a href="https://#question7">Question #7</a></li>
            <li><a href="https://#question8">Question #8</a></li>
            <li><a href="https://#question9">Question #9</a></li>
            <li><a href="https://#question10">Question #10</a></li>
            <li><a href="https://#question11">Question #11</a></li>
            <li><a href="https://#question12">Question #12</a></li>
        </ul>
</ol>

</div>


<h2>1. Import and Load Data </h2>


<h3>Import Libraries</h3>


Import the libraries necessary to use in this lab. We can add some aliases to make the libraries easier to use in our code and set a default figure size for further plots. Ignore the warnings.


In [ ]:
! conda install -c conda-forge ta-lib -y
!pip install --upgrade scikit-learn
!pip install mplfinance

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import seaborn as sns
from typing import List
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as  mpatches
%matplotlib inline
import mplfinance as fplt
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, accuracy_score, mean_absolute_percentage_error
import itertools
from itertools import combinations
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
import talib as tb

<h3> Load Data </h3>


We will use the same DataSet like in previous labs. Therefore next some steps will be the same.


First, we assign the URL of the dataset to <code>"path"</code>. 


In [ ]:
path = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0R9OEN/labs/MATICBUSD_trades_1m%20(2).csv"

Then use the Pandas method <code>read_csv()</code> to load the data from the web address and set dataframe index column type to <strong>datetime</strong> using <code>pd.to_datetime()</code> method for correct time series analysis. 


In [ ]:
df = pd.read_csv(path, index_col=0)
df.set_index('ts', inplace=True)
df.index = pd.to_datetime(df.index)

df.head()

<h2>2. Questions</h2>


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #1: </h1>

Calculate financial technical EMA, Crypto Trading Metrics, WMA.
    
</div>


<h4>Exponential Moving Average (EMA)</h4>


The exponential moving average (EMA) is a technical chart indicator that tracks the price of an investment (like a stock or commodity) over time. The EMA is a type of weighted moving average (WMA) that gives more weighting or importance to recent price data.

The $EMA$ is calculated as follows:

$$
EMA = Price(t) * k + EMA(y) * (1-k), 
$$
where t — today, y - yesterday, N - number of days in EMA, k =  2 / (N + 1)

In [ ]:
# Write your code below and press Shift+Enter to execute
ema = tb.EMA(df.avg_price, 64)
df['EMA'] = ema
ema[200:205]

<h4>WMA</h4>


The weighted moving average (WMA) is a technical indicator that assigns a greater weighting to the most recent data points, and less weighting to data points in the distant past.
The WMA is obtained by multiplying each number in the data set by a predetermined weight and summing up the resulting values.

$$
WMA = ∑ \frac {(price_i * (n-i))}{[n * (n + 1)] * 0.5},
$$

where n is a time period.
    


In [ ]:
# Write your code below and press Shift+Enter to execute
wma = tb.WMA(df.avg_price, 64)
df['WMA'] = wma
wma[200:205]

<h4>Commodity Channel Index (CCI)</h4>

The Commodity Channel Index (CCI) is an oscillator developed by Donald Lambert in 1980. It is based on the notion that as an equity tops or bottoms the volatility decreases. Lambert was struck by the fact that an equity can resemble a sine wave as it moves between highs and lows. The observation is obviously most valid when an equity is in a price channel.
$$
CCI = \frac {Typical price - MA}{MD * 0.015},
$$
where MA - Moving Average, MD - Mean Deviation 

In [ ]:
# Write your code below and press Shift+Enter to execute
cci = tb.CCI(df['high'], df['low'], df['close'])
# high_col='high', low_col='low', close_col='close'
df['CCI'] = wma
cci[200:205]

In [ ]:
# Write your code below and press Shift+Enter to execute
# Remove NaN values in dataframe 
df.dropna(inplace=True)
df

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #2: </h1>

Create new dataframe. Resample data for 1-day interval. Then plot a corresponding candlestick.
</div>


*  to keep the initial dataframe unchanged use <code>pd.DataFrame()</code> method for creating dataframe
*  use  method <code>.resample()</code> for resampling
*  to plot a candlestick use <code> fplt.plot(resampled_df,title='your title',ylabel='your y-label text',type='candle')</code>


In [ ]:
# Write your code below and press Shift+Enter to execute
# Resampling with 1-day interval
df_candle = pd.DataFrame()

df_candle['open'] = df['open'].resample('D').first()
df_candle['high'] = df['high'].resample('D').max()
df_candle['low'] = df['low'].resample('D').min()
df_candle['close'] = df['close'].resample('D').last()
df_candle['volume'] = df['volume'].resample('D').sum()
df_candle['rec_count'] = df['rec_count'].resample('D').mean()
df_candle['avg_price'] = df['avg_price'].resample('D').mean()
df_candle.head()

In [ ]:
# Write your code below and press Shift+Enter to execute
# Plot a candlestick
df_candle.columns = ['open', 'high', 'low', 'close', 'volume', 'rec_count', 'avg_price']
fplt.plot(
            df_candle,
            title='MATICBUSD',
            ylabel='Price (BUSD)',
            type='candle'
            
        )

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #3: </h1>

*    Create categorical variables( 3 categories : 'Low', 'Medium', 'High' ) for each of indicators and price.
*    Merge them to our dataframe.
*    Convert them to datatype <em>'category'</em>.
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute
# List of categories
group_names = ["Low", "Medium", "High"]

indicators = ['EMA', 'WMA', 'CCI']
for indicator in indicators:
    bins = np.linspace(min(df[indicator]), max(df[indicator]), 4)
    df[f"{indicator}-binned"] = pd.cut(df[indicator], bins, labels=group_names, include_lowest=True)
    
df.head()

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #4: </h1>

Find a correlation between MATIC price and obtained indicators. Build a heatmap.
</div>


*   use <code>df[[columns]].corr()</code> to calculate a correlation
*   use <code>seaborn.heatmap()</code> to visualize a correlation heatmap


In [ ]:
# Write your code below and press Shift+Enter to execute
# Find correlation
corr = df[['avg_price', 'EMA', 'WMA', 'CCI']].corr()
corr

In [ ]:
# Write your code below and press Shift+Enter to execute
# Build a heatmap
sns.heatmap(corr, annot=True, fmt=".3f")

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #5: </h1>
Plot a scatterplot with the fitted regression line for "EMA" indicator and MATIC price.
</div>


*    use <code>seaborn.regplot(x, y, data)</code> method to plot data and a linear regression model fit


In [ ]:
# Write your code below and press Shift+Enter to execute
sns.regplot(x=df['avg_price'], y=df['EMA'], data=pd)
plt.xlabel("EMA")
plt.ylabel("avg_price")
plt.show()


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #6: </h1>
For every indicator calculate(with avg_price):
*   calculate Pearson Correlation and P-value 
</div>


*   use <code>scipy.stats.pearsonr(x, y)></code> method for <em>Pearson Correlation</em> and <em>P-value</em> calculation


In [ ]:
# Write your code below and press Shift+Enter to execute
# Calculate Pearson Correlation and P-value for each of indicators and price 
df_stats = pd.DataFrame({"indicator":[], "Pearson": [], "P-value": []})
pd.options.display.float_format = '{:.3f}'.format

indicators = ['EMA', 'WMA', 'CCI']
for indicator in indicators:
    pearson_coef, p_value = stats.pearsonr(df['avg_price'], df[indicator])
    df_stats.loc[len(df_stats.index)] = [indicator, pearson_coef, p_value]

df_stats.set_index('indicator', inplace=True)
df_stats

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #7: </h1>

Create a Linear Regression model with any indicator as input and avg_price as output. Fit the model and make a prediction.

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute
# Create Multiple Linear Regression
lm = LinearRegression()

# Fit the model
Z = df[['CCI']]
lm.fit(Z, df['avg_price'])

# Make prediction
Y_hat = lm.predict(Z)
print(Y_hat)

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #8: </h1>

Plot Linear Regression.
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute
# Plot Multiple Linear Regression
plt.figure(figsize=(8, 6))

ax1 = sns.kdeplot(df['avg_price'],  color="tab:orange", label="avg_price")
sns.kdeplot(Y_hat, color="tab:blue", label="Fitted avg_price" , ax=ax1)

handles = [mpatches.Patch(facecolor=plt.cm.Reds(100), label="avg_price"),
           mpatches.Patch(facecolor=plt.cm.Blues(100), label="Fitted avg_price")]
plt.legend(handles=handles)

plt.title('Actual vs Fitted')
plt.xlabel('avg_price')
plt.ylabel('Proportion of cryptocurrency')

plt.show()
plt.close()

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #9: </h1>
For Linear Regression model calculate $R^2$, MSE metrics.
</div>


*   $R^2$ − <code>r2_score(y, y_pred)</code>
*   <em>MSE</em> − <code>mean_squared_error(y, y_pred)</code>


In [ ]:
# Write your code below and press Shift+Enter to execute
# Calculate metrics
y = df['avg_price']
r_squared = r2_score(y, Y_hat) *100
mse = mean_squared_error(y, Y_hat) *100
    
print(f'R-square = {r_squared:.3f}%')
print('')
print(f'MSE = {mse:.3f}%')

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #10: </h1>

Create "avg_price-binned". Encode categorical features as an integer array in order to use them in classification.
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute
bins = np.linspace(min(df["avg_price"]), max(df["avg_price"]), 4)
df["avg_price-binned"] = pd.cut(df["avg_price"], bins, labels=group_names, include_lowest=True)
# Encode categorical features
col_cat = ['EMA-binned', 'WMA-binned', 'CCI-binned']
X = df[col_cat]  #input columns
y = df['avg_price-binned']  #target column 


X = df[col_cat]
oe = OrdinalEncoder()
oe.fit(X[col_cat])
X_cat_enc = oe.transform(X[col_cat])

x_enc = pd.DataFrame(X_cat_enc)
x_enc.columns = col_cat

le = LabelEncoder()
le.fit(y)
y_enc = le.transform(y)
y_enc = pd.Series(y_enc)
y_enc.columns = y.name

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #11: </h1>

Split dataframe into train and test subsets.
</div>


* use <code>train_test_split(X, y, test_size)</code> to split the dataframe.


In [ ]:
# Write your code below and press Shift+Enter to execute
X_train, X_test, y_train, y_test = train_test_split(x_enc, y_enc, test_size=0.33, shuffle=False)

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #12: </h1>

Build a Decision Tree model and calculate its accuracy and plot it.
</div>


<em><strong>Note:</strong></em> use the following links to complete this question:
*   <code>DecisionTreeClassifier()</code> 
*   <code>accuracy_score()</code> to calculate accuracy
*   <code>plot_tree()</code>


In [ ]:
# Write your code below and press Shift+Enter to execute
# Decision Tree
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
yhat = model.predict(X_test)

accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % (accuracy*100), '%')

In [ ]:
# Write your code below and press Shift+Enter to execute
# Plot Decision Tree
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(model,
               feature_names = x_enc.columns, 
               class_names = y.unique(),
               filled = True)

# **Thank you for completing Lab 6 and our course at all!**
# **Good luck!**
## Authors

<a href="https://author.skills.network/instructors/oleh_lozovyi?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0QGDEN2306-2023-01-01">Oleh Lozovyi</a>

<a href="https://author.skills.network/instructors/yaroslav_vyklyuk_2?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0QGDEN2306-2023-01-01">Prof. Yaroslav Vyklyuk, DrSc, PhD</a>

<a href="https://author.skills.network/instructors/mariya_fleychuk?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0QGDEN2306-2023-01-01">Prof. Mariya Fleychuk, DrSc, PhD</a>

<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2021-01-01">Joseph Santarcangelo</a>


<a href="https://www.linkedin.com/in/fiorellawever/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2021-01-01">Fiorella Wenver</a>

<a href="https:// https://www.linkedin.com/in/yi-leng-yao-84451275/ " target="_blank" >Yi Yao</a>


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By   | Change Description                                         |
| ----------------- | ------- | -------------| ---------------------------------------------------------- |
|     2023-04-06    |   1.0   | Oleh Lozovyi | Lab created                                                |

<hr>

## <h3 align="center"> © IBM Corporation 2023. All rights reserved. <h3/>



